# Add Calculated Fields

## Overview
In this lesson we will download a small example TEEHR Evaluation from S3 and work through the different ways to add calculated fields to the joined_timeseries table, save them to the joined_timeseries table on disk to persist them, or add them temporarily before calculating metrics.  The first few steps for creating and cloning the TEEHR Evaluation should look familiar if you worked through the previous examples.

### Create a new Evaluation
First we will import TEEHR along with some other required libraries for this example.  Then we create a new instance of the Evaluation that points to a directory where the evaluation data will be stored.

In [ ]:
import teehr
from pathlib import Path
import shutil
from teehr.examples.setup_e0_2_example import download_e0_2_example

# Tell Bokeh to output plots in the notebook
from bokeh.io import output_notebook
output_notebook()

In [ ]:
# Define the directory where the Evaluation will be created
test_eval_dir = Path(Path().home(), "temp", "08_calculated_fields")
shutil.rmtree(test_eval_dir, ignore_errors=True)

download_e0_2_example(temp_dir=test_eval_dir)

# Create an Evaluation object and create the directory
ev = teehr.Evaluation(dir_path=Path(test_eval_dir, "e0_2_location_example"), create_dir=False)

## Add Calculated Fields to Joined Timeseries

Now we will get right to it.  Lets start by re-creating the `joined_timeseries` table and setting the `add_attrs=False` and `execute_udf=False` so that we get just the basic `joined_timeseries` table that joins the primary and secondary timeseries but does not add attributes or user defined fields. This step is not strictly necessary but makes it a bit easier to follow what we are doing in the subsequent steps.  Lets take a look at the `joined_timeseries` table and see what fields are included.

In [ ]:
ev.joined_timeseries.create(add_attrs=False, execute_scripts=False)
ev.joined_timeseries.to_sdf().show()

Ok, so, as shown, the `joined_timeseries` table now only contains the basic fields required without any attributes or user defined fields. Now lets add some fields to the `joined_timeseries` table using the TEEHR calculated fields.  Adding fields can be useful for grouping and filtering operations.  The TEEHR calculated fields (CF) allow for row level and timeseries aware CFs:

* Row level CFs are applied to each row in the table based on data that is in one or more existing fields.  These are applied per row and are not aware of the data in any other row (e.g., are not aware of any other timeseries values in a "timeseries").  This can be used for adding fields such as a field based on the data/time (e.g., month, year, season, etc.) or based on the value field (e.g., normalized flow, log flow, etc.) and many other uses.
* Timeseries aware CFs are aware of ordered groups of data (e.g., a timeseries).  This is useful for things such as event detection, base flow separation, and other fields that need to be calculated based on a entire collection of timeseries values.  The definition of what creates a unique set of timeseries (i.e., a timeseries) can be specified.

There are two ways that these CFs can be used:  

* First, they can be used to add the CF to the `joined_timeseries` which can then be persisted by writing to disk. This is useful if the calculation is expected to be needed for multiple different metric calculations.
* Second, they can be used as a pre-processing step in the calculation of metrics.

These use cases will be demonstrated below.  First we will import the CF classes.  Normally this would be done at the top of the page, but is done here for demonstration purposes.

In [ ]:
# first we need to import the UDF classes
from teehr import RowLevelCalculatedFields as rcf
from teehr import TimeseriesAwareCalculatedFields as tcf

### Available Calculated Fields
There are a number of calculated field classes that are included in the TEEHR package.  They are:

Row Level Calculated Fields:
- Month
- Year
- WaterYear
- NormalizedFlow
- Seasons
- ForecastLeadTime
- ForecastLeadTimeBins
- ThresholdValueExceeded
- DayOfYear

Timeseries Aware Calculated Fields.
- AbovePercentileEventDetection

There will be more added over time.  If there is one you are particularly interested in, please reach out and let us know.

### Add Calculated Field in Memory
Now we will use the Row level CFs to add year, month, water year and season to the `joined_timeseries` table in memory, using the `add_udf_columns()` method on the `joined_timeseries` table, but will not save it to disk yet.  Adding the UDFs and displaying the table shows that the new fields were added.

In [ ]:
sdf = ev.joined_timeseries.add_calculated_fields([
    rcf.Month(),
    rcf.Year(),
    rcf.WaterYear(),
    rcf.Seasons()
]).to_sdf()
sdf.show()

However, if we query the table again, we can see that the additional fields are not there.  This is because we did not write the table with the additional fields to disk.

In [ ]:
ev.joined_timeseries.to_sdf().show()

### Add Calculated Field and Persist
Now we will add the UDFs again, but this time we will write the table with the additional fields to disk so they are persisted.

In [ ]:
ev.joined_timeseries.add_calculated_fields([
    rcf.Month(),
    rcf.Year(),
    rcf.WaterYear(),
    rcf.Seasons()
]).write()


And query the table again.  This time we can see that the new fields we added are there as they were written to disk and are now part of the `joined_timeseries` table on disk.

In [ ]:
ev.joined_timeseries.to_sdf().show()

### Timeseries Aware Calculated Fields
The timeseries aware calculated fields behave the same way from the users perspective, but behind the scenes are performing some extra grouping and sorting to ensure that the field is calculated based on an ordered group of timeseries values (i.e., a "timeseries").  This is necessary for doing things like event detection, but comes at a computational cost, so use with care, especially on large datasets.  Lets try it.  This time we will jump right to writing the resulting data frame back to disk to persist it, but you could add the field and display the results without persisting as we did above.

In [ ]:
ev.joined_timeseries.add_calculated_fields([
    tcf.AbovePercentileEventDetection()
]).write()

And query the `joined_timeseries` table to see the new `event` and `event_id` fields.

In [ ]:
ev.joined_timeseries.to_sdf().show()

In the table above you can see the `event` and `event_id` fields, but it is a bit difficult to see what was done based on the table alone.  Lets create a plot to see what the new fields mean.  In the next few cells we will query the `joined_timeseries` table and filter the data to a single location ('usgs-14138800') and only values that were identified as `event=True`, then create a plot where we color the points by the new `event_id` field.

In [ ]:
import hvplot.pandas  # noqa

In [ ]:
pdf = ev.joined_timeseries.filter([
    "primary_location_id = 'usgs-14138800'",
    "event_above = true",
]).to_pandas()

In [ ]:
primary_plot = pdf.hvplot.points(x="value_time", y="primary_value", color="event_above_id") #.opts(width=1200, height=400)

In [ ]:
primary_plot.opts(width=1200, height=400)

If you zoom in on the plot above, you can see that each unique `event` has been given a unique `event_id` that was used to color the individual data points.  This was done by first identifying the 85 percentile value at each location and identifying each value as either above or below that value for its location.  Then the values were grouped by continuous sets of values that were identified as `event=True` and each continuous group was given an `event_id`.  This is just one example of what can be done using `TimeseriesAwareCalculatedFields`.

## Use Calculated Fields in Metrics

Beyond just identifying high flow events (in this example), the `event` and `event_id` fields can be used in subsequent metrics calculations.  In the next few cells we will demonstrate one way that the `event` and `event_id` fields can be used by working through the steps to calculate the `event_max_relative_bias`.  The `event_max_relative_bias` is the relative bias between the primary and secondary timeseries maximum values within each event.  We calculate this in twos steps using chained queries.  We will do it in two steps to demonstrate what we are doing. Note, if you have not worked though the grouping and filter notebooks you may want to go back and do that first as it is an important concept to understanding what is being done here. First we run a metrics query where we filter to a single location and only values that were identified as being `event=True`, group by `configuration_name`, `primary_location_id` and `event_id`, and calculate the maximum primary and secondary values which we call `max_primary_value` and `max_secondary_value` but could give them any name we wanted. 

In [ ]:
(
    ev.metrics
    .query(
        group_by=["configuration_name", "primary_location_id", "event_above_id"],
        filters=[
            "primary_location_id = 'usgs-14138800'",
            "event_above = true",
        ],
        include_metrics=[
            teehr.Signatures.Maximum(
                input_field_names=["primary_value"],
                output_field_name="max_primary_value"
            ),
            teehr.Signatures.Maximum(
                input_field_names=["secondary_value"],
                output_field_name="max_secondary_value"
            )
        ]
    )
    .to_sdf().show()
)

You can see that this gives us the `max_primary_value` and `max_secondary_value`  for each unique group of `configuration_name`, `primary_location_id` and `event_id`.  But, that is not what we are actually trying to calculate.  We are really trying to calculate the `event_max_relative_bias`.  To do that we have to add one more step by chaining together queries.  In the following query we add an additional chained query where we only group by `configuration_name` and `primary_location_id` which causes the query to aggregate the values from the different events, and then as an aggregation method (`include_metrics`) we choose relative bias.

In [ ]:
(
    ev.metrics
    .query(
        group_by=["configuration_name", "primary_location_id", "event_above_id"],
        filters=[
            "primary_location_id = 'usgs-14138800'",
            "event_above = true",
        ],
        include_metrics=[
            teehr.Signatures.Maximum(
                input_field_names=["primary_value"],
                output_field_name="max_primary_value"
            ),
            teehr.Signatures.Maximum(
                input_field_names=["secondary_value"],
                output_field_name="max_secondary_value"
            )
        ]
    )
    .query(
        group_by=["configuration_name", "primary_location_id"],
        include_metrics=[
            teehr.DeterministicMetrics.RelativeBias(
                input_field_names=["max_primary_value", "max_secondary_value"],
                output_field_name="event_max_relative_bias"
            )
        ]
    )
    .to_sdf().show()
)

One last thing to cover here.  So far we have added the calculated fields on the `joined_timeseries` table, written them to disk, and then queried the `joined_timeseries` table to calculate metrics.  This works well and allows the calculated fields to be calculated once and used in many subsequent metrics, plots, etc.  However, you may wish to add temporary fields to the `joined_timeseries` table as part of the metrics calculation.  This can be done too.  Building on the previous example, where we calculated the "event_max_relative_bias", lets now assume we want to calculate the same metric but for the 90th percentile instead of the default 85th percentile that we used when we added the added the `event` and `event_id` fields to the `joined_timeseries` table.  We could add the new "90th percentile event" to the `joined_timeseries` table and save to disk and then proceed as we did before, or we can add new `event90` and `event90_id` fields to the data frame temporarily before calculating the maximum event values and ultimately the "event_90th_max_relative_bias".

In [ ]:
(
    ev.metrics
    # Add the AbovePercentileEventDetection calculated field to identify events greater than the 90th percentile.
    # Note the output_event_field_name and output_event_id_field_name are set to "event90" and "event90_id" respectively.
    .add_calculated_fields([
        tcf.AbovePercentileEventDetection(
            quantile=0.90,
            output_event_field_name="event90",
            output_event_id_field_name="event90_id"
        )
    ])
    # First query to calculate the maximum primary and secondary values for each event.
    # Note the filters are set to only include events where event90 is true and the group_by includes event90_id.
    .query(
        group_by=["configuration_name", "primary_location_id", "event90_id"],
        filters=[
            "primary_location_id = 'usgs-14138800'",
            "event90 = true",
        ],
        include_metrics=[
            teehr.Signatures.Maximum(
                input_field_names=["primary_value"],
                output_field_name="max_primary_value"
            ),
            teehr.Signatures.Maximum(
                input_field_names=["secondary_value"],
                output_field_name="max_secondary_value"
            )
        ]
    )
    # Second query to calculate the relative bias between the maximum primary and secondary values.
    .query(
        group_by=["configuration_name", "primary_location_id"],
        include_metrics=[
            teehr.DeterministicMetrics.RelativeBias(
                input_field_names=["max_primary_value", "max_secondary_value"],
                output_field_name="event_90th_max_relative_bias"
            )
        ]
    )
    # Convert the metrics to a pandas DataFrame
    .to_sdf().show()
)

Categorical Deterministic Metrics
---------------------------------

Now that we understand routines for grouping/filtering data and added calculated fields in TEEHR to obtain performance metrics, we can introduce the concept of categorical deterministic metrics. 

Categorical deterministic metrics are utilized to measure qualitative attributes and are used to evaluate binary outcomes given some condition or categorical classification. Put simply, these methods help compare predictions made by the model with observed data to indicate where the model was right or wrong.

Currently, the following categorical deterministic methods are available in TEEHR:

- [ConfusionMatrix](https://rtiinternational.github.io/teehr/api/generated/teehr.DeterministicMetrics.html#teehr.DeterministicMetrics.ConfusionMatrix)
- [FalseAlarmRatio](https://rtiinternational.github.io/teehr/api/generated/teehr.DeterministicMetrics.html#teehr.DeterministicMetrics.FalseAlarmRatio)
- [ProbabilityOfDetection](https://rtiinternational.github.io/teehr/api/generated/teehr.DeterministicMetrics.html#teehr.DeterministicMetrics.ProbabilityOfDetection)
- [ProbabilityOfFalseDetection](https://rtiinternational.github.io/teehr/api/generated/teehr.DeterministicMetrics.html#teehr.DeterministicMetrics.ProbabilityOfFalseDetection)
- [CriticalSuccessIndex](https://rtiinternational.github.io/teehr/api/generated/teehr.DeterministicMetrics.html#teehr.DeterministicMetrics.CriticalSuccessIndex)

Unlike other metrics in TEEHR which operate on the default fields present in the joined timeseries table, categorical deterministic metrics require an additional 'threshold field' column to categorize model predictive performance based on a user defined flow threshold. 

For example, let's utilize our categorical metrics to evaluate model performance when predicting streamflow that exceeds the 10% return interval. Lets start from the default `joined_timeseries` table and complete the chained operation in-memory as opposed to writing to disk or using the fields we calculated in the previous steps.

In [ ]:
# overwrite the existing joined_timeseries table to demonstrate chained query
ev.joined_timeseries.create(add_attrs=False, execute_scripts=False)

In [ ]:
metrics_df = ev.metrics.add_calculated_fields([
    # adds 'event', 'event_id', and 'quantile_value' fields to the joined timeseries table
    teehr.TimeseriesAwareCalculatedFields.AbovePercentileEventDetection(
        quantile=0.90,
        add_quantile_field=True
    )
]).query(
    # calculate all available categorical deterministic metrics using the 'quantile_value' field as the threshold
    group_by=['primary_location_id', 'configuration_name'],
    include_metrics=[
        teehr.DeterministicMetrics.ConfusionMatrix(
            threshold_field_name='quantile_value'
        ),
        teehr.DeterministicMetrics.FalseAlarmRatio(
            threshold_field_name='quantile_value'
        ),
        teehr.DeterministicMetrics.ProbabilityOfDetection(
            threshold_field_name='quantile_value'
        ),
        teehr.DeterministicMetrics.ProbabilityOfFalseDetection(
            threshold_field_name='quantile_value'
        ),
        teehr.DeterministicMetrics.CriticalSuccessIndex(
            threshold_field_name='quantile_value'
        )
    ]
).to_sdf().show()

In the above example, we utilize a chained query to add a 'threshold field' to our unaltered `joined_timeseries` table and calculate all available categorical metrics in one operation. 

The chained operation executes in two steps:

- The first portion of the operation uses the `AbovePercentileEventDetection` method with `add_quantile_field` set to True to obtain the flow value that corresponds to the 90% quantile for each row in the joined timeseries table.

- The following portion takes the resulting table (with `AbovePercentileEventDetection` fields present) and executes a metrics query for our categorical deterministic metrics (with `threshold_field_name` arguments populated with the default `output_quantile_field_name` specified in the `AbovePercentileEventDetection` method). 

As you can see, we obtain a unique entry for the following metrics per unique combination of `primary_location_id` and `configuration_name` (as specified in the `group_by` argument in the `ev.metrics.query()` call).

<b>When employing categorical deterministic metrics in TEEHR, keep in mind that each aggregation must correspond to exactly one 'threshold value'</b>. 

In this example, that criteria is achieved inherently given the `quantile_value` was generated using a `TimeSeriesAwareCalculatedField` which considers each 'unique timeseries' (where 'unique timeseries' is defined by `AbovePercentileEventDetection.uniqueness_fields`, by default=`['reference_time', 'primary_location_id', 'configuration_name', 'variable_name', 'unit_name']`). 

By that same logic, each unique combination of `primary_location_id` and `configuration_name` defined in the `group_by` argument in this example has exactly one 'threshold value' because `['reference_time', 'variable_name', 'unit_name']` fields are constant for each unique combination of `['primary_location_id, 'configuration_name']`.

In [ ]:
ev.spark.stop()